In [128]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
import pickle
from valorant import *
import plotly.graph_objects as go

In [129]:
def get_region(team):
    if team in emea:
        return "EMEA"
    elif team in amer:
        return "AMERICAS"
    elif team in apac:
        return "PACIFIC"
    else:
        return "OTHER"


In [138]:
span = "all"
with open(f"data/data-{span}.pkl", "rb") as f:
    data = pickle.load(f)

In [139]:
df = []
for i in data.values():
    columns = {"name": i["name"], "team": i["team"], "rating": i["rating"]}
    for agent in agent_roles.keys():
        if agent in i["stats"].index:
            columns[agent] = i["stats"].loc[agent, "Usage count"]
        else:
            columns[agent] = 0
    df.append(columns)
df = pd.DataFrame(df)
df["region"] = df["team"].apply(lambda x: get_region(x))
df = df[df["region"] != "OTHER"]

In [140]:
tsne = UMAP(n_neighbors=7, min_dist=0.1, metric="euclidean", random_state=42)
agents = df[agent_roles.keys()].T.values
X = tsne.fit_transform(agents / agents.sum(axis=0))
fig = px.scatter(
    x=X[:, 0],
    y=X[:, 1],
    text=agent_roles.keys(),
    # symbol=agent_roles.values(),
    color=agent_roles.values(),
    size_max=60,
    size=agents.sum(axis=1),
    width=700,
    height=700,
)
fig.update_layout(
    coloraxis_colorbar=dict(x=1.1, title="Usage count"),
    legend=dict(orientation="h", yanchor="bottom", y=0, xanchor="left", x=0, title="Role"),
    margin=dict(l=10, r=10, b=10, t=30),
    title=f"UMAP plot of agent usage, span={span}",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
)
fig.write_html(f"figs/agents-{span}.html")
fig.write_image(f"figs/agents-{span}.png", scale=2)
fig.show()

In [141]:
def get_annotation(row):
    annotation = f"<b>{row['name']}</b><br>Team: {row['team']}<br>Rating: {row['rating']:.2f}<br><br>"
    maps_played = row[agent_roles.keys()].sum()
    for agent, pickrate in row[agent_roles.keys()].sort_values(ascending=False).to_dict().items():
        if pickrate:
            annotation += f"{agent}: {pickrate/maps_played:.2f}<br>"
    return annotation

In [142]:
tsne = UMAP(n_neighbors=20, min_dist=0.7, metric="euclidean", random_state=42)
# tsne = PCA(n_components=2, random_state=42)
# tsne = TSNE(n_components=2, random_state=42)
df["annotation"] = df.apply(get_annotation, axis=1)
df["favorite_agent"] = df[agent_roles.keys()].idxmax(axis=1)
val = df[list(agent_roles.keys())].values
X = tsne.fit_transform(val)
df["x"] = X[:, 0]
df["y"] = X[:, 1]
fig = px.scatter(
    df,
    x="x",
    y="y",
    hover_name="name",
    text="name",
    color="favorite_agent",
    # symbol="region",
    custom_data=["annotation"],
    width=900,
    height=700,
)
# fig.update_layout(coloraxis_colorbar=dict(x=1.1))
fig.update_layout(
    coloraxis_colorbar=dict(x=1.1, title="Rating"),
    # legend=dict(orientation="h", yanchor="bottom", y=0, xanchor="left", x=0, title="Region", tracegroupgap=10),
    margin=dict(l=10, r=10, b=10, t=30),
    title=f"UMAP plot of player agent picks, span={span}",
    xaxis_title="Feature 1",
    yaxis_title="Feature 2",
)
fig.update_traces(
    marker=dict(
        size=12,
        opacity=0.8,
    ),
    textposition="top center",
    hovertemplate="%{customdata}",
)

fig.write_html(f"figs/players-{span}.html")
fig.write_image(f"figs/players-{span}.png", scale=2)

fig.show()